In [4]:
import os
from langsmith.run_trees import RunTree

os.environ["LANGCHAIN_PROJECT"] = "pycon23"

In [5]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI()
llm.predict("Hello, world!")

'Hello! How can I assist you today?'

permitted run_types: [tool', 'chain', 'llm', 'retriever', 'embedding', 'prompt', 'parser']

In [11]:
parent_run = RunTree(
    name="My Chat Bot",
    run_type="chain",
    inputs={"text": "Summarize this morning's meetings."},
    serialized={}
)

child_llm_run = parent_run.create_child(
    name="My Proprietary LLM",
    run_type="retriever",
    inputs={
        "prompts": [
            "You are an AI Assistant. Summarize this morning's meetings."
        ]
    },
)

child_llm_run.end(outputs={"generations": ["Summary of the meeting..."]})
parent_run.end(outputs={"output": ["The meeting notes are as follows:..."]})

res = parent_run.post(exclude_child_runs=False)
res.result()

In [7]:
parent_run.id

UUID('2e128808-9206-41dd-a6f6-03661ca8f972')

In [16]:
from langsmith import Client

client = Client()
client.create_feedback(
    parent_run.id,
    "human_feedback",
    value = "like",
    feedback_source_type="model"
)

Feedback(id=UUID('9a5b6387-00b9-4e79-a93b-baea9275331a'), created_at=datetime.datetime(2023, 9, 28, 10, 9, 28, 350759, tzinfo=datetime.timezone.utc), modified_at=datetime.datetime(2023, 9, 28, 10, 9, 28, 350766, tzinfo=datetime.timezone.utc), run_id=UUID('d96edc39-e89d-4d55-8bd4-32cd5b7b268f'), key='human_feedback', score=None, value='like', comment=None, correction=None, feedback_source=FeedbackSourceBase(type='model', metadata={}))